【Point】
- マイニングに時間がかかる場合は、INITIAL_BITSの値を0x1e777777に変更してみましょう。

In [1]:
# 問1

import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1d777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        new_bits = self.get_retarget_bits()
        if new_bits < 0:
            bits = last_block.bits
        else:
            bits = new_bits
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), bits)
        self.mining(block)

    def get_retarget_bits(self):
        if len(self.chain) == 0 or len(self.chain) % 5 != 0:
            return -1
        
        expected_time = 140 * 5
        
        if len(self.chain) != 5:
            first_block = self.chain[-(1 + 5)]
        else:
            first_block = self.chain[0]
        
        last_block = self.chain[-1]
        
        first_time = first_block.timestamp.timestamp()
        last_time = last_block.timestamp.timestamp()
        
        total_time = last_time - first_time
        target = last_block.calc_target()
        
        delta = total_time / expected_time
        if delta < 0.25:
            delta = 0.25
        if delta > 4:
            delta = 4
        
        new_target = int(target * delta)
        exponent_bytes = (last_block.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        temp_bits = new_target >> exponent_bits
        
        if temp_bits != temp_bits & 0xffffff:
            exponent_bytes += 1
            exponent_bits += 8
        elif temp_bits == temp_bits & 0xffff:
            exponent_bytes -= 1
            exponent_bits -= 8
        return ((exponent_bytes + 3) << 24) | (new_target >>exponent_bits)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "000000212bc07eface48ae8edb7c54ec49341df8ce2ef5c0765af2319f452bbf",
  "elapsed_time": "25.389秒",
  "index": 0,
  "nonce": "0063bfaa",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2019/10/29 11:29:40"
}
2番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "000000206d9441f082f84f5a9efcca74ff1d427edfb1bf598b6e4af9dbbd8e55",
  "elapsed_time": "13.123秒",
  "index": 1,
  "nonce": "00325f60",
  "prev_hash": "000000212bc07eface48ae8edb7c54ec49341df8ce2ef5c0765af2319f452bbf",
  "stored_data": "ブロック 1",
  "timestamp": "2019/10/29 11:30:06"
}
3番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000002460820cbe139e7c8e5724598ea85364ad0850ac37997d7cb784b5fb76",
  "elapsed_time": "39.976秒",
  "index": 2,
  "nonce": "009a5925",
  "prev_hash": "000000206d9441f082f84f5a9efcca74ff1d427edfb1bf598b6e4af9dbbd8e55",
  "stored_data": "ブロック 2",
  "timestamp": "20

{
  "bits": "1d802357",
  "block_hash": "0000007ef9693660720e9fe6aeb2d55adda7db0997c8277e752e8e4951b5d692",
  "elapsed_time": "77.671秒",
  "index": 24,
  "nonce": "0122e456",
  "prev_hash": "00000022eeaa1f2e89b2630a5c6e0f807b159ab2f51dfa9ef4055021900c6ff7",
  "stored_data": "ブロック 24",
  "timestamp": "2019/10/29 12:59:27"
}
26番目のブロックを作成中・・・
{
  "bits": "1d6d045b",
  "block_hash": "00000066e6ff4ed7446f955bd3d1965ca8f4fb2ac543726a008c3f8df824d333",
  "elapsed_time": "110.915秒",
  "index": 25,
  "nonce": "01975148",
  "prev_hash": "0000007ef9693660720e9fe6aeb2d55adda7db0997c8277e752e8e4951b5d692",
  "stored_data": "ブロック 25",
  "timestamp": "2019/10/29 13:00:45"
}
27番目のブロックを作成中・・・
{
  "bits": "1d6d045b",
  "block_hash": "0000002f81fcd345ae980204be39b21a7a20f65850faa0ccb5e250be6a85a869",
  "elapsed_time": "109.383秒",
  "index": 26,
  "nonce": "0198cf77",
  "prev_hash": "00000066e6ff4ed7446f955bd3d1965ca8f4fb2ac543726a008c3f8df824d333",
  "stored_data": "ブロック 26",
  "timestamp": "2019/10/29 1

In [15]:
# 問2

import hashlib
import datetime
import time
import json
import random

INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

# SHA256を用いたハッシュ化を関数として扱いやすくしています
def sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = sha256(blockheader)
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class MerkleTree():
    def __init__(self):
        self.tree_path = []
        f = open("/Users/akazawanaoki/Documents/dev/mempool.json", "r")
        mempool = json.load(f) # jsonデータとして変数mempoolに格納しています
        tx_list = mempool["tx"] # 全部のトランザクションのリストを取得しています
        c = random.randint(2, 30) # 2から30までの整数の乱数を生成しています
        txs_in_this_block = random.sample(tx_list, c) # ブロックに取り込まれるTxをランダムにダブりなしで取得しています
        self.tree_path.append(txs_in_this_block) # ブロックに取り込むTxのリストをツリーの最初に追加しています
        f.close()

    def calc_merkleroot(self):
        txs = self.tree_path[0]
        if len(txs) == 1:
            return txs[0]
        while len(txs) > 1:
            if len(txs) % 2 == 1:
                txs.append(txs[-1])
            hashes = []
            for i in range(0, len(txs), 2):
                hashes.append(sha256("".join(txs[i:i + 2])))
            txs = hashes
        return txs[0]

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        mt = MerkleTree()
        merkleroot = mt.calc_merkleroot()
        block = Block(i+1, last_block.block_hash, merkleroot, datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00003da3de11463491fd1516b3576091e95cb2ddcea5295a07a210c2357aee6e",
  "elapsed_time": "2.096秒",
  "index": 0,
  "nonce": "0004ad5d",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2019/10/28 18:06:12"
}
2番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000036ff6fdb56acf179d240398c4716c4671b4a40b7486ee7396badafaa3879",
  "elapsed_time": "2.072秒",
  "index": 1,
  "nonce": "0004edf9",
  "prev_hash": "00003da3de11463491fd1516b3576091e95cb2ddcea5295a07a210c2357aee6e",
  "stored_data": "2a37a061c200ade8c98e556985c8be5f96d59f8dd57cb4574170be90b467f98b",
  "timestamp": "2019/10/28 18:06:15"
}
3番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000011df5c9be6f4795a7b1026011ca64e28eee0e7c91f9a3735d9ad42b0b3a6",
  "elapsed_time": "1.44秒",
  "index": 2,
  "nonce": "00037f89",
  "prev_hash": "000036ff6fdb56acf179d240398c4716c4671b4a40b7486ee7396badaf

{
  "bits": "1e777777",
  "block_hash": "00006f06260a752191f4e01824c9d5312c04a63b084278e2cb40aeea5f04b67c",
  "elapsed_time": "0.244秒",
  "index": 21,
  "nonce": "0000941c",
  "prev_hash": "0000619d6b5ba215605c2458d11ca652bf3ebb6c1a60a78c6e229e8aacc4eab7",
  "stored_data": "d3a0790337a80c248695aeb48d721851f33c7f9fb0a48284991f78fcf443669f",
  "timestamp": "2019/10/28 18:06:37"
}
23番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000066aad50e65869f657993f6a78b9eea8be383a131e1dec5aae52bfd26ceec",
  "elapsed_time": "0.91秒",
  "index": 22,
  "nonce": "000236f5",
  "prev_hash": "00006f06260a752191f4e01824c9d5312c04a63b084278e2cb40aeea5f04b67c",
  "stored_data": "9307051bb62adba609491d98aa8c90cac88b51196d6733b6e0d161ed834e547a",
  "timestamp": "2019/10/28 18:06:37"
}
24番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000062b7f2878c4b2519f4b5340fc91555afdd109fc3a0096301dc24dcf36784",
  "elapsed_time": "0.623秒",
  "index": 23,
  "nonce": "00017caf",
  "prev_hash": "000066aad50e658

In [18]:
# 問3

import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1d777777
MAX_32BIT = 0xffffffff

# SHA256を用いたハッシュ化を関数として扱いやすくしています
def sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = sha256(blockheader)
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

# マークルルートを計算するMerkleTreeクラスを実装しています
class MerkleTree():
    def __init__(self):
        self.tree_path = []
        f = open("/Users/akazawanaoki/Documents/dev/mempool.json", "r")
        mempool = json.load(f) # jsonデータとして変数mempoolに格納します
        tx_list = mempool["tx"] # 全部のトランザクションのリストを取得します
        c = random.randint(2, 30) # 2から30までの整数の乱数を生成します
        txs_in_this_block = random.sample(tx_list, c) # ブロックに取り込まれるTxをランダムにダブりなしで取得します
        self.tree_path.append(txs_in_this_block) # ブロックに取り込むTxのリストをツリーの最初に追加します
        f.close()

    def calc_merkleroot(self):
        txs = self.tree_path[0]
        if len(txs) == 1:
            return txs[0]
        while len(txs) > 1:
            if len(txs) % 2 == 1:
                txs.append(txs[-1])
            hashes = []
            for i in range(0, len(txs), 2):
                hashes.append(sha256("".join(txs[i:i + 2])))
            txs = hashes
        return txs[0]

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        new_bits = self.get_retarget_bits()
        if new_bits < 0:
            bits = last_block.bits
        else:
            bits = new_bits
        #マークルルートを計算するためにMerkleTreeクラスをインスタンス化し、calc_merklerootメソッドを利用してmerklerootを算出しています。
        mt = MerkleTree()
        merkleroot = mt.calc_merkleroot()
        
        block = Block(i+1, last_block.block_hash, merkleroot, datetime.datetime.now(), bits)
        self.mining(block)
        
    # 難易度調整のメソッドです。
    def get_retarget_bits(self):
        if len(self.chain) == 0 or len(self.chain) % 5 != 0:
            return -1
        
        expected_time = 140 * 5
        
        if len(self.chain) != 5:
            first_block = self.chain[-(1 + 5)]
        else:
            first_block = self.chain[0]
        
        last_block = self.chain[-1]
        
        first_time = first_block.timestamp.timestamp()
        last_time = last_block.timestamp.timestamp()
        total_time = last_time - first_time
        target = last_block.calc_target()
        
        delta = total_time / expected_time
        if delta < 0.25:
            delta = 0.25
        if delta > 4:
            delta = 4
        
        new_target = int(target * delta)
        exponent_bytes = (last_block.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        temp_bits = new_target >> exponent_bits
        
        if temp_bits != temp_bits & 0xffffff:
            exponent_bytes += 1
            exponent_bits += 8
        elif temp_bits == temp_bits & 0xffff:
            exponent_bytes -= 1
            exponent_bits -= 8
        return ((exponent_bytes + 3) << 24) | (new_target >>exponent_bits)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "00000051f55b2a5fef39bc6d2ebff5c5fe425cf63cf63740c3bb97b37136e9cd",
  "elapsed_time": "695.226秒",
  "index": 0,
  "nonce": "069e72b1",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2019/10/28 18:18:16"
}
2番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000002fc57550f7113f626acb6a56a83c421318552e400793ccd57c7683f4ce",
  "elapsed_time": "73.063秒",
  "index": 1,
  "nonce": "00bf0dcf",
  "prev_hash": "00000051f55b2a5fef39bc6d2ebff5c5fe425cf63cf63740c3bb97b37136e9cd",
  "stored_data": "36a75f8da0e54c22138fbf1c86b3f4d11a254055bd744cebcc3a24ce450c4b62",
  "timestamp": "2019/10/28 18:29:51"
}
3番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "00000018a341ba0be2342888df98edb3efb61882bcdb541d51b5d646abcc04b0",
  "elapsed_time": "1981.561秒",
  "index": 2,
  "nonce": "01a28050",
  "prev_hash": "0000002fc57550f7113f626acb6a56a83c421318552e400793c

{
  "bits": "1e01354f",
  "block_hash": "000000926243f706fd55db818ce9498c76c60363d7b747b7a371d80629bfbc29",
  "elapsed_time": "49.354秒",
  "index": 21,
  "nonce": "0093e81a",
  "prev_hash": "000000e0b04d33d22515523c4a0db6adea4af40e6e0f244c9a90d6360749a1c4",
  "stored_data": "198b16842ac21937dd6899709e1e7def9b9267eb0f3cdbec1f06604d50a956ff",
  "timestamp": "2019/10/28 21:03:24"
}
23番目のブロックを作成中・・・
{
  "bits": "1e01354f",
  "block_hash": "0000010962d7f865c43be4b39e1cf6507675749914ecd0df36858224c6d7d961",
  "elapsed_time": "47.91秒",
  "index": 22,
  "nonce": "0090c30c",
  "prev_hash": "000000926243f706fd55db818ce9498c76c60363d7b747b7a371d80629bfbc29",
  "stored_data": "bc17e969bf0751530bc9f2ebd3315b8118db39caabeb8d2ff795afb23988bb73",
  "timestamp": "2019/10/28 21:04:13"
}
24番目のブロックを作成中・・・
{
  "bits": "1e01354f",
  "block_hash": "00000033b316dcbe4936fdcd8339a1c9b83d83f24e7baf7386c1c8f776011c2c",
  "elapsed_time": "1763.397秒",
  "index": 23,
  "nonce": "00880fac",
  "prev_hash": "0000010962